In [4]:
import numpy as np
import scipy.special
import matplotlib.pyplot as plt
import scipy.special # 시그모이드 함수를 사용하기 위해 사용

class NeuralNetwork:
    # 신경망 초기화
    def __init__(self, inputnodes, hiddennodes, outputnodes, learnrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # 가중치 행렬 w input -> hidden, w hidden -> output
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        self.lr = learnrate

        # 활성화 함수 지정
        self.activation_func = lambda x: scipy.special.expit(x)
        # activation_func(5) == sigmoid(5)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # 신경망 학습
    def train(self, inputs_list, targets_list):
        # 1. 입력 리스트를 2차원 행렬로 변환
        inputs = np.array(inputs_list, ndmin=2).T # ndmin=2 차수를 2차원으로 변경
        targets = np.array(targets_list, ndmin=2).T
        # 2. 은닉계층으로 들어오는 값 계산 (input -> hidden)
        hidden_inputs = np.dot(self.wih, inputs)
        # 3. 은닉계층에서 나가는 값 계산 (hidden -> output) -> activation_func(==sigmoid)
        hidden_outputs = self.sigmoid(hidden_inputs)
        # 4. 최종계층으로 들어오는 값 계산 (hidden -> output)
        final_inputs = np.dot(self.who, hidden_outputs)
        # 5. 최종계층에서 나가는 값 계산
        final_outputs = self.sigmoid(final_inputs)
        # 6. 오차 계산 (실제 값 - 모델에 의해서 계산된 값)
        output_errors = targets - final_outputs
        # 7. 오차역전법 (은닉계층의 오차는 가중치에 의해 나뉜 출력계층의 오차들의 재조합)
        hidden_errors = np.dot(self.who.T, output_errors) # 179p
        # 8. 은닉계층과 출력계층 간 가중치 업데이트
        # new w(eight) = old w - lr x (dE/dw = (dE/do:은닉계층에서 재조합 된 역전파오류 * do/dnet:은닉계층의 노드 j로 들어오는 입력 값에 가중치를 적용한 결과 * dnet/dw: 첫번째 계층의 노드 oi의 결과 값))
        self.who += self.lr * np.dot((output_errors * final_outputs * (1-final_outputs)), np.transpose(hidden_outputs))
        # 9. 입력계층과 은닉계층 간 가중치 업데이트
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1-hidden_outputs)), np.transpose(inputs))
    
    # 신경망 검증(테스트)
    def test(self, inputs_list):
        # 1. 입력리스트를 2차원 행렬로 변환
        inputs = np.array(inputs_list, ndmin=2).T
        # 2. 은닉계층으로 들어오는 값 계산
        hidden_inputs = np.dot(self.wih, inputs)
        # 3. 은닉계층에서 나가는 값 계산
        hidden_outputs = self.sigmoid(hidden_inputs)
        # 4. 최종계층으로 들어오는 값 계산
        final_inputs = np.dot(self.who, hidden_outputs)
        # 5. 최종계층에서 나가는 값 계산
        final_outputs = self.sigmoid(final_inputs)

        return final_outputs

In [5]:
input_nodes = 784
hidden_nodes = 100
output_nodes = 10
learning_rate = 0.3

model = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# 학습 데이터(샘플 100) 파일 읽어오기
train_data_file = open('mnist_train.csv', 'r')
train_data_list = train_data_file.readlines()
train_data_file.close()

for record in train_data_list:
    # record 값을 [,]로 구분
    values = record.split(',')
    # 값 조정 -> X(기존 값) / 255.0 * 0.99 + 0.01
    inputs = (np.asfarray(values[1:])/255.0 * 0.99) + 0.01
    # target 리스트 준비 (10개 짜리 빈 list -> one hot encoding)
    targets = np.zeros(output_nodes) + 0.01
    # 정답을 one hot encoding으로 표시
    targets[int(values[0])] = 0.99
    model.train(inputs, targets)
    pass


# epochs = 10
# for e in range(epochs):
#     for record in train_data_list:
#         # record 값을 [,]로 구분
#         values = record.split(',')
#         # 값 조정 -> X(기존 값) / 255.0 * 0.99 + 0.01
#         inputs = np.asfarray(values[1:])/255.0 * 0.99 + 0.01
#         # target 리스트 준비 (10개 짜리 빈 list -> one hot encoding)
#         targets = np.zeros(output_nodes) + 0.01
#         # 정답을 one hot encoding으로 표시
#         targets[int(values[0])] = 0.99
#         model.train(inputs, targets)
#         pass
#     pass

In [6]:
test_data_file = open('mnist_test.csv', 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

values = test_data_list[0].split(',')
print(values[1])
print(model.test(np.asfarray(values[1:])/(255.0 * 0.99) + 0.01))

0
[[0.01490283]
 [0.01648703]
 [0.02738006]
 [0.03859851]
 [0.00618534]
 [0.0107662 ]
 [0.0132307 ]
 [0.97576434]
 [0.00681003]
 [0.00281338]]


In [7]:
scorecard = []
for record in train_data_list:
    values = record.split(',')
    correct_label = int(values[0])
    # print('correct label=', correct_label)

    inputs = (np.asfarray(values[1:])/255.0 * 0.99) + 0.01
    outputs = model.test(inputs)
    predict = np.argmax(outputs)
    # print('model predict=', predict)
    
    if predict == correct_label:
        scorecard.append(1)
    else:
        scorecard.append(0)

scorecard_array = np.asfarray(scorecard)    
print('accuracy_score=', scorecard_array.sum()/ scorecard_array.size)

accuracy_score= 0.9460666666666666


In [ ]:
for record in train_data_list:
    values = record.split(',')
    image = np.asfarray(values[1:]).reshape((28,28))
#     plt.imshow(image, cmap='Greys', interpolation=None)
#     plt.show()

In [ ]:
print(image)

# NUMPY

In [ ]:
# numpy의 행렬 곱이 필요한 이유!!

A = np.array([[1,2], [3,4]])
B = np.array([10, 20])
print(np.dot(A,B)) # 행렬 곱!!
print('A=',A, '\nB=', B)

In [ ]:
# numpy 행렬 변환
A = []
for i in range(10):
    A.append(i+1)
B = np.array(A)
C = np.array(A, ndmin=2).T # 가로로 된 행렬을 세로형의 2차원 배열로 바꿔서 입력값을 바꿔줌
print(A)
print(B)
print(C)